<center>

<h1 style="text-align:center"> Streams, Laziness and Memoization </h1>
<h2 style="text-align:center"> CS3100 Fall 2019 </h2>
</center>

## Review

### Previosly

* Modular Programming
  + Namespacing, Abstraction, Code Reuse
  + Structures, Signatures, Functors
  
### This lecture

* Streams: Programming with infinite data structures
* Laziness: Call-by-need evaluation

## Recursive values

* In OCaml, we can define recursive functions.
  + we can also define **recursive values**

In [ ]:
(* Infinite list of ones *)
let rec ones = 1::ones

In [ ]:
(* Infinite list of alternating 0s and 1s *)
let rec zero_ones = 0::1::zero_ones

Even though the list is **infinite**, the data structure uses **finite** memory. 

## Infinite data structures

Infinite data structures are not just an intellectual curiosity.

* Infinite sequences such as primes and fibonacci numbers.
* Streams of input read from file or socket.
* Game trees which may be infinite
  + Every possible move leads to branch in the tree. 
  + Imagine game trees where a piece could chase the other around forever. 

## Limitations of cyclic structures

Suppose we want to convert the infinite list `zero_ones` to string, the obvious solutions don't work. 

In [ ]:
let zero_ones_string = List.map string_of_int zero_ones

## List to Streams

We can start with the list type

```ocaml
type 'a list = Nil | Cons of 'a * 'a list
```

and make a **stream** type.

In [ ]:
type 'a stream = Cons of 'a * 'a stream

There is no `Nil` since the streams are infinite. 

## Doesn't quite work

In [ ]:
let rec zero_ones = Cons (0, Cons (1, zero_ones))

In [ ]:
let rec to_string (Cons(x,xs)) = Cons(string_of_int x, to_string xs)

In [ ]:
to_string zero_ones

## Pausing the execution

* We need a way to pause the execution rather than recursively applying to the rest of the list. 
* Use **thunks**: `unit -> 'a` functions.

In [ ]:
let v = failwith "error"

## Pausing the execution

In [ ]:
let f = fun () -> failwith "error"

In [ ]:
f ()

## Streams again

In [ ]:
type 'a stream = Cons of 'a * (unit -> 'a stream)

In [ ]:
let rec zero_ones = Cons (0, fun () -> Cons (1, fun () -> zero_ones))

In [ ]:
let hd (Cons (x, _)) = x

In [ ]:
let tl (Cons (_, xs)) = xs ()

## More Stream functions

In [ ]:
let rec take n s = 
  if n = 0 then []
  else hd s::(take (n-1) (tl s))

In [ ]:
take 10 zero_ones

In [ ]:
let rec drop n s =
  if n = 0 then s
  else drop (n-1) (tl s)

In [ ]:
drop 1 zero_ones

## Higher order functions on streams

In [ ]:
let rec map f s = Cons (f (hd s), fun () -> map f (tl s))

In [ ]:
let zero_ones_str = map string_of_int zero_ones

In [ ]:
take 10 zero_ones_str

## Higher order functions on streams

In [ ]:
(** [filter p s] returns a new stream where every element [x] in [s] 
    such that [p x = true] is removed *)
let rec filter p s =
  if p (hd s) then filter p (tl s)
  else Cons (hd s, fun () -> filter p (tl s))

In [ ]:
let s' = filter ((=) 0) zero_ones in
take 10 s'

## Higher order functions on streams

In [ ]:
let rec zip f s1 s2 = Cons (f (hd s1) (hd s2), fun () -> zip f (tl s1) (tl s2))

In [ ]:
zip (fun x y -> (x,y)) zero_ones zero_ones_str

## Primes

* **Sieve of Eratosthenes**: Neat way to compute primes.
* Start with a stream `s` of `[2;3;4;.....]`.
* At each step, 
  + `p = hd s` is a prime.
  + return a new stream `s'` such that $\forall x.x \text{ mod } p \notin s'$
* In the first step,
  + `prime = 2`
  + `new stream = [3;5;7;9;11;13;15;17;....]`
* In the second step,
  + `prime = 3`
  + `new stream = [5;7;11;13;17;19;23;....]`

## Primes

In [ ]:
let rec from n = Cons (n, fun () -> from (n+1));;
from 2

In [ ]:
let primes_stream =
  let rec primes s = Cons (hd s, fun () -> 
    primes @@ filter (fun x -> x mod (hd s) = 0) (tl s))
  in primes (from 2)

In [ ]:
take 10 @@ primes_stream

## Fibonacci sequence

* Let's consider Fibonacci sequence
  + `s1 = [1;1;2;3;5;8;13;...]`
* Let's consider the tail of `s1`
  + `s2 = [1;2;3;5;8;13;....]`
* Let's zip `s1` and `s2` by adding together the elements:
  + `s3 = [2;3;5;6;13;21;...]`
  + `s3` is nothing but the tail of tail of fibonacci sequence. 
* If we were to prepend `[1;1]` to `s3` we will have the fibonacci sequence.



## Fibonacci sequence

In [ ]:
let rec fibs = 
  Cons (1, fun () -> 
    Cons (1, fun () -> 
      zip (+) fibs (tl fibs)))

In [ ]:
take 10 fibs

## Fibonacci sequence

* Each time we force the computation of the next element, we compute the fibonacci of previous element twice.
  + Not immediately apparent, but this is equivalent to:

```ocaml
let rec fib n = if n < 2 then 1 else fib (n-1) + fib (n-2)
```

There is an exponential increase in the running time of `fib(n)` for each increase in `n`.

## Lazy Values

* It would be nice to **save** the results of the execution for previously seen values and reuse them.
  + This is the idea behind lazy values in OCaml.
* Lazy values are the opt-in, explicit, call-by-name reduction strategy for OCaml
  + Rest of the language is strict i.e, call-by-value
* Lazy module in OCaml is:

```ocaml
module Lazy = 
  type 'a t = 'a lazy_t
  val force : 'a t -> 'a
```

OCaml has syntactic support for lazy values through the `lazy` keyword.

## Lazy values

In [ ]:
let v = lazy (10 + (print_endline "Hello"; 20))

In [ ]:
Lazy.force v

In [ ]:
Lazy.force v

## Lazy fib

In [ ]:
let fib30lazy = lazy (take 30 fibs |> List.rev |> List.hd)

In [ ]:
Lazy.force fib30lazy

In [ ]:
let fib29lazy = take 31 fibs |> List.rev |> List.hd

## Lazy stream

Let's redefine the stream using lazy values. 

In [ ]:
type 'a stream = Cons of 'a * 'a stream Lazy.t

In [ ]:
let hd (Cons (x,l)) = x
let tl (Cons (x,l)) = Lazy.force l
let rec take n s = 
  if n = 0 then [] else hd s::(take (n-1) (tl s))
let rec zip f s1 s2 = 
  Cons (f (hd s1) (hd s2), lazy (zip f (tl s1) (tl s2)))

## Fibs Lazy Streams

In [53]:
let rec fibslazystream = 
  Cons (1, lazy (
    Cons (1, lazy (
      zip (+) fibslazystream (tl fibslazystream)))))

error: compile_error

In [ ]:
take 30 fibslazystream

You can see that this is fast!

## Memoization

* Lazy values in OCaml are a specific efficient implementation of the general idea of caching called **Memoization**.
  + Add caching to functions to retrieve results fast. 

In [54]:
let memo f = 
  let cache = Hashtbl.create 16 in
  fun v -> 
    match Hashtbl.find_opt cache v with
    | None -> 
        let res = f v in 
        Hashtbl.add cache v res;
        res
    | Some res -> res

val memo : ('a -> 'b) -> 'a -> 'b = <fun>


## Expensive identity

In [55]:
let rec spin n = if n = 0 then () else spin (n-1)

val spin : int -> unit = <fun>


In [56]:
let expensive_id x = spin 200000000; x

val expensive_id : 'a -> 'a = <fun>


In [59]:
expensive_id 10

- : int = 10


## Memoizing expensive identity

In [60]:
let memoized_expensive_id = (memo expensive_id)

val memoized_expensive_id : '_weak1 -> '_weak1 = <fun>


In [62]:
memoized_expensive_id 11

- : int = 11


## Memoizing recursive functions

* Memoizing recursive functions is a bit more tricky.
  + We need to tie the **recursive knot**

In [63]:
let rec fib n = 
  if n < 2 then 1 else fib(n-2) + fib(n-1)

val fib : int -> int = <fun>


In [65]:
fib 40

- : int = 165580141


## Memoizing recursive functions

Simply doing `let memo_fib = memo fib` will only memoize the outer calls and not the recursive calls.

In [67]:
let memo_fib = memo fib

val memo_fib : int -> int = <fun>


In [73]:
memo_fib 40

- : int = 165580141


## Tying the recursive knot

This function should remind you of the definition we used for Y combinator.

In [74]:
let fib_norec f n = if n < 2 then 1 else f (n-1) + f(n-2)

val fib_norec : (int -> int) -> int -> int = <fun>


The idea is to provide an `f` which is the memoized version of 

```ocaml
let rec f n = if n < 2 then 1 else f (n-1) + f(n-2)
```

We will use a **reference** to tie the knot.

## Tying the recursive knot

`memo_rec` will memoize recursive function that take an explicit recursive function argument such as `fib_norec`.

In [80]:
let memo_rec f_norec =
  (* define a reference [f] to a function which will never be invoked *)
  let f = ref (fun _ -> assert false) in
  (* memoize the "eta-expanded" [f_norec] function by dereferencing [f]. *)
  let f_rec_memo = memo (fun x -> f_norec !f x) in
                                      (* [f] is not dereferenced yet *)
  f := f_rec_memo; (* update [f] to the recursive memoized function *)
  f_rec_memo

val memo_rec : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


In [81]:
let fib_memo = memo_rec fib_rec

val fib_memo : int -> int = <fun>


In [83]:
fib_memo 30

- : int = 1346269


<center>

<h1 style="text-align:center"> Fin. </h1>
</center>